# Explore and process the South America gravity data

This notebook loads and processes the raw gravity data in [../data/goco05s-sam-s0.1deg-h50km.gdf](../data/goco05s-sam-s0.1deg-h50km.gdf). The data are gravity values in mGal. This notebook calculates the gravity disturbance, performs topographic correction, calculates and removes the effect of sediments (taken from the CRUST1.0 model).

The results are saved to text files in the `data` folder for use in inversion and making figures.

## Package imports

In [ ]:
%matplotlib inline

Load the standard scientific Python stack to numerical analysis and plotting.

In [ ]:
from __future__ import division
import datetime
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import multiprocessing
import seaborn  # Makes the default style of the plots nicer

The computations generate a lot of run-time warnings. They aren't anything to be concerned about so disable them to avoid clutter.

In [ ]:
import warnings
warnings.simplefilter('ignore')

Load the required modules from [Fatiando a Terra](http://www.fatiando.org).

In [ ]:
from fatiando.gravmag import tesseroid, normal_gravity
from fatiando import gridder, utils
import fatiando

In [ ]:
print("Version of Fatiando a Terra used: {}".format(fatiando.__version__))

Load our custom classes and functions.

In [ ]:
from mohoinv import TesseroidRelief, make_mesh
from datasets import fetch_crust1, load_icgem_gdf, down_sample

Get the number of cores in this computer to run the some things in parallel.

In [ ]:
ncpu = multiprocessing.cpu_count()
print("Number of cores: {}".format(ncpu))

## Load and plot the raw data

In [ ]:
data = load_icgem_gdf('../data/goco05s-sam-s0.1deg-h50km.gdf')

The data is return in a Python dictionary. The following fields are read from the file:

In [ ]:
print(data.keys())

`metadata` is the file header.

In [ ]:
print(data['metadata'])

We'll need to down sample this data set to a larger grid spacing because it's too large for modeling.

In [ ]:
arrays = data['latitude'], data['longitude'], data['height'], data['gravity_ell']
downsample_every = 2
lat, lon, height, grav, shape = down_sample(arrays, data['shape'], every=downsample_every)
area = (lat.min(), lat.max(), lon.min(), lon.max())
print("Data area (S, N, W, E): {}".format(area))
print("Number of points in latitude and longitude: {}".format(shape))
print("Original dataset size: {}".format(data['shape']))

Setup a [basemap](http://matplotlib.org/basemap/index.html) to plot the data with an appropriate projection.

In [ ]:
bm = Basemap(projection='cyl', 
             llcrnrlon=area[2], urcrnrlon=area[3], llcrnrlat=area[0], urcrnrlat=area[1],
             lon_0=0.5*(area[2] + area[3]), lat_0=0.5*(area[1] + area[0]), 
             resolution='l')

Make a plotting function to avoid repeating this code.

In [ ]:
def plot_data(lat, lon, data, shape, cmap, cblabel='mGal', levels=60, ranges=True):
    x, y = bm(lon, lat) # Transform lat and lon into plot coordinates
    kwargs = dict(cmap=cmap)
    if ranges:
        ranges = np.abs([data.min(), data.max()]).max()
        kwargs['vmin'] = -ranges
        kwargs['vmax'] = ranges
    fig = plt.figure(figsize=(7, 6))
    bm.contourf(x.reshape(shape), y.reshape(shape), data.reshape(shape), levels, **kwargs)
    plt.colorbar(pad=0.01, aspect=50).set_label(cblabel)
    bm.drawmeridians(np.arange(-80, -30, 10), labels=[0, 0, 0, 1], linewidth=0.2)
    bm.drawparallels(np.arange(-50, 30, 15), labels=[1, 0, 0, 0], linewidth=0.2)
    bm.drawcoastlines(color="#333333")
    plt.tight_layout(pad=0)
    return fig

Plot the raw gravity data.

In [ ]:
plot_data(lat, lon, grav, shape, 'Reds', ranges=False)
plt.title('Raw Gravity')

## Calculate gravity disturbance

The gravity distance is the raw gravity data minus the effect of the Normal Earth ($\gamma$) calculated at the observation point.

Fatiando a Terra offers the `gamma_closed_form` function that calculates $\gamma$ at any latitude and height using the closed form formula in [Li and Gotze (2001)](http://dx.doi.org/10.1190/1.1487109).

In [ ]:
disturbance = grav - normal_gravity.gamma_closed_form(lat, height)

In [ ]:
plot_data(lat, lon, disturbance, shape, 'RdBu_r')
plt.title('Gravity disturbance')

## Terrain correction

The next step is to remove the effect of the topography and water layer. Let's load the topography data downloaded from [ICGEM](http://icgem.gfz-potsdam.de/ICGEM/). This data is [ETOPO1](http://ngdc.noaa.gov/mgg/global/global.html) calculated by interpolation on the specified grid points of our data.

In [ ]:
topo_data = load_icgem_gdf('../data/topography-sam-s0.1deg.gdf', usecols=[-1])

In [ ]:
print(topo_data['metadata'])

In [ ]:
topo, _ = down_sample([topo_data['topography_grd']], topo_data['shape'], every=downsample_every)

In [ ]:
plot_data(lat, lon, topo, shape, cmap='terrain', cblabel='meters')
plt.title('Topography')

We'll make a tesseroid model of the topography so that we can calculate it's gravitational effect in spherical coordinates. The `make_mesh` function of [`mohoinv.py`](mohoinv.py) automates this process for us. Each point in the topography grid is at the center of the top face of a tesseroid.

In [ ]:
topo_model = make_mesh(area, shape, topo, reference=0) 

Now we need to set a density value for the topography. We'll use the standard 2670 kg/m³ for the rocks (topography > 0) and -1630 = 1040 - 2670 kg/m³ for water (topography < 0).

In [ ]:
topo_density = 2670*np.ones(topo_model.size)
# Density in the oceans is rho_water
topo_density[topo_model.relief < topo_model.reference] = -1630
topo_model.addprop('density', topo_density)

Forward model the effect of the topography in spherical coordinates using tesseroids. Use all available cores for this calculation. It will take a while.

In [ ]:
%time topo_effect = tesseroid.gz(lon, lat, height, topo_model, njobs=ncpu)

In [ ]:
plot_data(lat, lon, topo_effect, shape, cmap='RdBu_r')
plt.title('Topographic gravitational effect')

The Bouguer anomaly will be the disturbance minus the effect of the topography.

In [ ]:
bouguer = disturbance - topo_effect

In [ ]:
plot_data(lat, lon, bouguer, shape, cmap='RdBu_r')
plt.title('Bouguer anomaly')

## Remove the effect of sediments


We need to remove the gravitational effect of the sedimentary layers from our data to isolate the Moho effect. We'll not consider any other crutal density anomalies because South America is not well represented in the CRUST1.0 model. Instead of assuming a most likely wrong crustal density, we choose to err on the side of simplicity and assume no crustal density anomalies.

Load the CRUST1.0 model for South America.

In [ ]:
crust1 = fetch_crust1('../data/crust1.0.tar.gz').cut(area)

Get the three sedimentary layers from the model. We need to transform their density values into density contrasts with respect to the standard crustal density od 2670 kg/m³.

In [ ]:
layers = [l.contrast('density', 2670) 
          for l in [crust1.upper_sediments, crust1.middle_sediments, crust1.lower_sediments]]
layer_names = ['Upper sediments', 'Middle Sediments', 'Lower sediments']

Plot the sediment thickness.

In [ ]:
x, y = bm(crust1.lon + 360, crust1.lat) # Transform lat and lon into plot coordinates
for layer_name, layer in zip(layer_names, layers):
    fig = plt.figure(figsize=(7, 6))
    bm.pcolormesh(x, y, layer.thickness, cmap='copper')
    plt.colorbar(pad=0.01, aspect=50).set_label('Thickness (m)')
    bm.drawmeridians(np.arange(-80, -30, 10), labels=[0, 0, 0, 1], linewidth=0.2)
    bm.drawparallels(np.arange(-50, 30, 15), labels=[1, 0, 0, 0], linewidth=0.2)
    bm.drawcoastlines(color="#eeeeee")    
    plt.title(layer_name)
    plt.tight_layout(pad=0)

Calculate the gravitational effect of each layer and the total effect of all layers.

In [ ]:
%time sediment_effects = [tesseroid.gz(lon, lat, height, list(layer.tesseroids), njobs=ncpu) for layer in layers]

In [ ]:
total_sediment_effect = np.sum(sediment_effects, axis=0)

Plot the effect of each layer and the total sediment effect.

In [ ]:
for layer_name, effect in zip(layer_names, sediment_effects):
    plot_data(lat, lon, effect, shape, 'RdBu_r')
    plt.title("Gravitational effect of " + layer_name.lower())

In [ ]:
plot_data(lat, lon, total_sediment_effect, shape, cmap='RdBu_r')
plt.title('Total sediment effect')

Calculate the sediment-free Bouguer anomaly.

In [ ]:
sedfree_bouguer = bouguer - total_sediment_effect

In [ ]:
plot_data(lat, lon, sedfree_bouguer, shape, cmap='RdBu_r')
plt.title('Sediment-free Bouguer anomaly')

## Save the data

Save the processed data and all intermediate values to the space-delimited text file [../data/processed-goco5s-data-sam-h50km.txt](../data/processed-goco5s-data-sam-h50km.txt).

In [ ]:
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
header = """# Generated by process-sam-gravity-data.ipynb on {date}
# shape (nlat, nlon):
# {nlat} {nlon}
# lat lon height topo gravity disturbance topo_effect bouguer upper_sediment middle_sediment lower_sediment total_sediment sedfree_bouguer
""".format(date=now, nlat=shape[0], nlon=shape[1])
with open('../data/processed-goco5s-data-sam-h50km.txt', 'w') as f:
    f.write(header)
    np.savetxt(f, np.c_[lat, lon, height, topo, grav, disturbance, topo_effect, bouguer, 
                        sediment_effects[0], sediment_effects[1], sediment_effects[2], total_sediment_effect, 
                        sedfree_bouguer],
               fmt='%.5f')